In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
from bs4 import BeautifulSoup as bs
import re
import requests

import warnings
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

### Парсинг вакансий с hh.ru

In [20]:
#Задаем параметры поиска
vacancy_name = 'data scientist'
search_depth = 6 #количество страниц поиска
analysis_period = 15 #дни публикаций
#area = 1(Moscow), 4(Novosibirsk),1001(Other countries)

In [21]:
#Получаем ссылку для выполнения поиска
hh_url = 'https://hh.ru/search/vacancy?search_period='+str(analysis_period)+'&text='+vacancy_name+'&area=1&area=4&area=1001'
hh_url

'https://hh.ru/search/vacancy?search_period=15&text=data scientist&area=1&area=4&area=1001'

In [10]:
#Обход блокировки hh.ru от автоматического парсинга
headers = {'accept': '*/*',
           'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15'}

session = requests.session()

In [22]:
n = 0
titles = []
employer = []
address = []
salarys = []
hrefs = []
description = []
requir = []

while n < search_depth:
    request = session.get(hh_url+'&page='+str(n), headers=headers)
    
    hh_html = bs(request.content, 'html.parser')
    divs = hh_html.find_all('div', attrs={'data-qa': 'vacancy-serp__vacancy'})

    for div in divs:
        title = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}).text
        company = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-employer'}).text
        place = div.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-address'}).text
        city = (re.findall('[а-яА-Я]+', place))[0]
        salary = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'})
        responsibility = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy_snippet_responsibility'}).text
        requirment = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy_snippet_requirement'}).text

        if salary == None:
            salary = 'Не указанно'
        else:
            salary = div.find('div', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}).text
        
        href = div.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})['href']
        
        titles.append(title)
        employer.append(company)
        salarys.append(salary)
        address.append(city)
        hrefs.append(href)
        description.append(responsibility)
        requir.append(requirment)
        
        #print(f'Вакансия: {title} в {company}\nПредлагаемая зарплата: {salary}\nСсылка: {href}\n{place}')
        #print(f'{responsibility}\n{requirment}\n\n')
        #print(city)
    
    n += 1

In [23]:
#Данные зальем в датафрейм
df = pd.DataFrame({'Title': titles,
                   'Company': employer,
                   'Salary': salarys,
                   'Address': address,
                   'Description': description,
                   'Requirments': requir,
                   'Href': hrefs}, columns=['Title','Company', 'Salary','Address','Description','Requirments', 'Href'])

df.replace(u'\xa0',u'', regex=True, inplace=True)
df.head()

,Title,Company,Salary,Address,Description,Requirments,Href
0,"Data Scientist (Кипр, Лимассол)",Кадровое агентство Алексея Сухорукова,3500-5000 EUR,Новосибирск,Работа с большими объемами структурированных и...,Опыт разработки масштабируемых математических ...,https://novosibirsk.hh.ru/vacancy/32375692?que...
1,Главный аналитик-математик (Data Analyst / Dat...,Связной,180000-230000 руб.,Москва,Исследование данных методами математической ст...,Хорошее владение языком R (предпочтительно) ил...,https://novosibirsk.hh.ru/vacancy/30250856?que...
2,Data scientist,Свордфиш Секьюрити,от 150000 руб.,Москва,Извлечение данных из нескольких информационный...,Hadoop как плюс. Postrgre SQL MySQL. Java как ...,https://novosibirsk.hh.ru/vacancy/32586327?que...
3,Data scientist,Билайн,Не указанно,Москва,Под руководством Senior Data Scientist и совме...,Знание основных видов математических моделей и...,https://novosibirsk.hh.ru/vacancy/32667835?que...
4,Senior Data Scientist (Академгородок),ООО ЭЛСИ ГРУПП,100000-200000 руб.,Новосибирск,Распознавание образов в видеопотоке. Улучшение...,"Понимание статистики и теории вероятностей, ум...",https://novosibirsk.hh.ru/vacancy/29701878?que...


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 7 columns):
Title          118 non-null object
Company        118 non-null object
Salary         118 non-null object
Address        118 non-null object
Description    118 non-null object
Requirments    118 non-null object
Href           118 non-null object
dtypes: object(7)
memory usage: 6.5+ KB


In [25]:
df['Address'].unique()

array(['Новосибирск', 'Москва', 'Финляндия'], dtype=object)

In [30]:
#df.groupby('Salary', as_index=False).size()

rcParams['figure.figsize'] = 8, 3
df.groupby('Salary', as_index=True).size().plot(kind='bar', title='График количества вакансий и зарплаты')

In [26]:
#Выгрузка базы в файл
writer = pd.ExcelWriter('ds.xlsx')
df.to_excel(writer, 'Сведения по вакансии')
writer.save()

In [27]:
#Выборка определенного города
nsk = df.loc[(df['Address']=='Новосибирск'),:]
nsk

,Title,Company,Salary,Address,Description,Requirments,Href
0,"Data Scientist (Кипр, Лимассол)",Кадровое агентство Алексея Сухорукова,3500-5000 EUR,Новосибирск,Работа с большими объемами структурированных и...,Опыт разработки масштабируемых математических ...,https://novosibirsk.hh.ru/vacancy/32375692?que...
4,Senior Data Scientist (Академгородок),ООО ЭЛСИ ГРУПП,100000-200000 руб.,Новосибирск,Распознавание образов в видеопотоке. Улучшение...,"Понимание статистики и теории вероятностей, ум...",https://novosibirsk.hh.ru/vacancy/29701878?que...
7,Data scientist,Центр финансовых технологий,Не указанно,Новосибирск,Прогнозирование спроса. Идентификация пользова...,"Опыт работы с данными в проектах, желательно п...",https://novosibirsk.hh.ru/vacancy/30318660?que...
11,Data Scientist (Академгородок),ООО ЭЛСИ ГРУПП,Не указанно,Новосибирск,Системы кредитного скоринга для различных стра...,"Хорошая математическая подготовка, знание осно...",https://novosibirsk.hh.ru/vacancy/31150346?que...
15,Data Scientist,Кадровое агентство Алексея Сухорукова,80000-200000 руб.,Новосибирск,Анализ поведения клиентов для решения задач ма...,Опыт в коммерческих/не коммерческих проектах п...,https://novosibirsk.hh.ru/vacancy/31812383?que...
74,Специалист по анализу данных (Data Scientist),Auriga,Не указанно,Новосибирск,Создание (в команде) новых сценариев использов...,Уверенное знание инструментов статистического ...,https://novosibirsk.hh.ru/vacancy/32591451?que...
84,Data scientist,ООО ВилайнТв,от 70000 руб.,Новосибирск,Умеете систематизировать полученную информацию...,Умеете читать и интерпретировать данные. Имеет...,https://novosibirsk.hh.ru/vacancy/31106608?que...
105,Data Scientist (Санкт-Петербург),ЦРТ | Группа компаний,Не указанно,Новосибирск,Разработка алгоритмов распознавания речи/ эмоц...,Хорошая математическая подготовка (знание мето...,https://novosibirsk.hh.ru/vacancy/22856468?que...
